In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.circuit.library import MCMT

import math
from bitstring import BitArray

# the following function builds a gate (oracle) that transforms |x>|y> to |x>|y - cut(x)>
def energy_oracle(graph_laplacian: list[list[int]], digits: int):

    # setting up the circuit
    qregx = QuantumRegister(len(graph_laplacian),"x")
    qregy = QuantumRegister(digits,"y")
    qc = QuantumCircuit(qregx,qregy)

    # QFT
    for i in range(digits):
        qc.h(qregy[i])

        theta = math.pi/2
        for j in range(i + 1, digits):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    # Phasers
    for i in range(len(graph_laplacian)):
        theta = - graph_laplacian[i][i] * math.pi
        for j in range(digits):
            qc.cp(theta, qregx[i], qregy[j])
            theta /= 2

        for j in range(i + 1, len(graph_laplacian[i])):
            if graph_laplacian[i][j] != 0:
                theta = - graph_laplacian[i][j] * math.pi * 2
                for k in range(digits):
                    qc.mcp(theta, [qregx[i], qregx[j]], qregy[k])
                    theta /= 2

    # Inverse QFT
    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    for i in range(digits - 1, - 1, - 1):
        qc.h(qregy[i])

        theta = - math.pi/2
        for j in range(i - 1, - 1, - 1):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    return qc.to_gate()



# the following function builds the G(alpha, beta) gates for the fixed point search
def grover_bangbang(graph_laplacian: list[list[int]], digits: int, alpha: float, beta: float):
    
    U_f = energy_oracle(graph_laplacian, digits)
    U_f.label = "U_f"
    U_f_inv = U_f.inverse()
    U_f_inv.label = "U_f_inv"

    qregx = QuantumRegister(len(graph_laplacian), "x")
    qregy = QuantumRegister(digits, "y")

    grover_bangbang = QuantumCircuit(qregx,qregy)

    # S_t (beta)
    grover_bangbang.append(U_f, qregx[:] + qregy[:])
    grover_bangbang.p(beta, qregy[0])
    grover_bangbang.append(U_f_inv, qregx[:] + qregy[:])



    # S_s (alpha)
    grover_bangbang.h(qregx)
    grover_bangbang.x(qregx)
    grover_bangbang.mcp(beta, qregx[:-1], qregx[-1])
    grover_bangbang.x(qregx)
    '''
    grover_bangbang.p(- alpha / 2, qregx[-1])

    grover_bangbang.mct(qregx[:-1], qregx[-1])
    grover_bangbang.mct(qregx[:-1], qregy[0])

    grover_bangbang.p(- alpha / 2, qregx[-1])
    grover_bangbang.p(- alpha / 2, qregy[0])

    grover_bangbang.mct(qregx[:-1], qregx[-1])
    grover_bangbang.mct(qregx[:-1], qregy[0])

    grover_bangbang.p(alpha, qregx[-1])
    '''
    grover_bangbang.h(qregx)

    # gateify 
    GBB = grover_bangbang.to_gate()
    GBB.label = "GBB"

    return GBB



#graph_laplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
#digits = 4
#grover_bangbang(graph_laplacian, digits, math.pi, 3)



# creates the S_L circuit
def grover_fixed_point_search(graph_laplacian: list[list[int]], digits: int, l: int, delta: float):

    alpha = []

    gamma = 1 / (math.cosh(math.acosh(1 / delta) / (2 * l + 1)))
    sq_gamma = math.sqrt(1 - gamma * gamma)
    L = 2 * l + 1

    for j in range(1, l + 1):        
        arg = sq_gamma * math.tan(math.pi * j / L)
        alpha.append(2 * math.atan(1 / arg))

    qregx = QuantumRegister(len(graph_laplacian), "x")
    qregy = QuantumRegister(digits, "y")

    qc = QuantumCircuit(qregx,qregy)

    qc.h(qregx)

    for j in range(l):
        G_j = grover_bangbang(graph_laplacian, digits, alpha[j], - alpha[l - 1 - j])
        G_j.label = "G(alpha_" + str(j + 1) + ", beta_" + str(j + 1) + ")"
        qc.append(G_j, qregx[:] + qregy[:])

    GFPS = qc.to_gate()
    GFPS.label = "GFPS"

    return GFPS



# TEST

graph_laplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
digits = 4

# the threshold y = 2; 1/8 of all configurations are above it

y_bits = [0, 0, 1, 0]
y = 2

l = 12 # one more than needed
delta = 0.01

GFPS = grover_fixed_point_search(graph_laplacian, digits, l, delta)

# one quantum-classical register pair for each vertex
qregx = QuantumRegister(len(graph_laplacian), "x")
cregx = ClassicalRegister(len(graph_laplacian), "cl-x")
# one um-classical register pair for each digit
qregy = QuantumRegister(digits, "y")
cregy = ClassicalRegister(digits, "cl-y")

qcircuit = QuantumCircuit(qregx,qregy,cregx,cregy)

for i in range(len(qregx)):
    qcircuit.initialize([1, 0], qregx[i])

for i in range(len(qregy)):
    qcircuit.initialize([1 - y_bits[i], y_bits[i]], qregy[i])

qcircuit.append(GFPS, qregx[:] + qregy[:])

# add one more energy oracle to see the cut values
U_f = energy_oracle(graph_laplacian, digits)
U_f.label = "energy"
qcircuit.append(U_f, qregx[:] + qregy[:])

# measure
qcircuit.measure(qregx, cregx)
qcircuit.measure(qregy, cregy)


print(qcircuit.draw())

simulator = QasmSimulator()
shot_number = 100
compiled_qcircuit = transpile(qcircuit, simulator)
job = simulator.run(compiled_qcircuit, shots=shot_number)
result = job.result()
counts = result.get_counts(compiled_qcircuit)


print(counts)

# results

adjusted_counts = []

for s in counts:
    adjusted_counts.append([s[::-1], 100*counts[s]/shot_number])

adjusted_counts.sort()

for s in adjusted_counts:
    b = s[0][-4:]
    cut = BitArray(bin=b).int
    cut = y - cut
    print("configuration =",s[0][:4],"result =",b," cut =",cut," frequency =",s[1],"%")


        ┌─────────────────┐┌───────┐┌─────────┐┌─┐                     
   x_0: ┤ Initialize(1,0) ├┤0      ├┤0        ├┤M├─────────────────────
        ├─────────────────┤│       ││         │└╥┘┌─┐                  
   x_1: ┤ Initialize(1,0) ├┤1      ├┤1        ├─╫─┤M├──────────────────
        ├─────────────────┤│       ││         │ ║ └╥┘┌─┐               
   x_2: ┤ Initialize(1,0) ├┤2      ├┤2        ├─╫──╫─┤M├───────────────
        ├─────────────────┤│       ││         │ ║  ║ └╥┘┌─┐            
   x_3: ┤ Initialize(1,0) ├┤3      ├┤3        ├─╫──╫──╫─┤M├────────────
        ├─────────────────┤│  GFPS ││  energy │ ║  ║  ║ └╥┘┌─┐         
   y_0: ┤ Initialize(1,0) ├┤4      ├┤4        ├─╫──╫──╫──╫─┤M├─────────
        ├─────────────────┤│       ││         │ ║  ║  ║  ║ └╥┘┌─┐      
   y_1: ┤ Initialize(1,0) ├┤5      ├┤5        ├─╫──╫──╫──╫──╫─┤M├──────
        ├─────────────────┤│       ││         │ ║  ║  ║  ║  ║ └╥┘┌─┐   
   y_2: ┤ Initialize(0,1) ├┤6      ├┤6        ├─╫──╫──╫──╫──╫──╫